# TensorFlow 모델 저장 복구 튜토리얼

- 어떻게 모델은 보이는가
- 어떻게 모델은 저장 되는가
- prediction과 transfer learning을 위한 restoe 방법
- pre-trained model을 불러오는 방법 fine-tuning을 위해서

In [4]:
import tensorflow as tf

# Prepare to feed input, i.e. feed_dict and placeholders
w1 = tf.placeholder(tf.float32, name="w1")
w2 = tf.placeholder(tf.float32, name="w2")
b1 = tf.Variable(2.0,dtype=tf.float32, name="bias")
feed_dict = {'w1': 4.0, 'w2': 8.0}

# Define a test operation that we will restore
w3 = w1 + w2
w4 = tf.multiply(w3, b1, name="op_to_restore")
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Create a saver object which will save all the variables
saver = tf.train.Saver()

# Run the operation by feeding input
result = sess.run(w4, {w1:feed_dict['w1'], w2:feed_dict['w2']})
print(result)
# Prints 24 which is sum of (w1+w2)*b1

# Now, save the graph
saver.save(sess, './my_test_model', global_step=1000)


24.0


'./my_test_model-1000'

복구를 위해서는
- graph 
- variable

모두를 restore 해야 한다.  
이것을 통해서 새로운 training data를 `feed`할 수 있다.  
`graph.get_tensor_by_name()` method를 통해서 saved operation과 placeholder variables의 reference를 얻을 수 있다.  

만약 같은 네트워크에 다른 data를 넣어서 처리하고 싶다면 간단하게 feed_dict을 네트워크에 넣는다.

In [8]:
import tensorflow as tf

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('my_test_model-1000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))


# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

print (sess.run(op_to_restore,feed_dict))
#This will print 60 which is calculated 
#using new values of w1 and w2 and saved value of b1. 

INFO:tensorflow:Restoring parameters from ./my_test_model-1000
60.0


만약 operation과 layer를 좀 더 추가하고 싶다면 아래와 같이 수행 한다.

In [10]:
import tensorflow as tf

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('my_test_model-1000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))


# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

#Add more to the current graph
add_on_op = tf.multiply(op_to_restore,2)

print (sess.run(add_on_op,feed_dict))
#This will print 120.

INFO:tensorflow:Restoring parameters from ./my_test_model-1000
120.0


만약 fine-tuning을 위해서 일부분만 저장하고 복구하고 싶다면?  
`graph.get_tensor_by_name()` method를 이용해서 적절한 operation만 접근할 수 있다.